In [ ]:
%%bash
pip install matplotlib aiohttp

# 安装中文字体
yum install wqy-zenhei-fonts wqy-microhei-fonts fontconfig
# 检查字体是否安装成功
fc-list :lang=zh
fc-cache


In [ ]:
%%bash
# 安装python依赖
#pip install grpcio grpcio-tools requests
#pip install numpy matplotlib

# 生产grpc的proto文件
cat > StockNotify.proto <<EOF
syntax = "proto3";

service Greeter {
  // 定义一个简单的 RPC 方法
  rpc StockNotify (StockRequest) returns (StockReply);
}

// 定义请求和响应消息
message StockRequest {
  string stock_code = 1;
  string stocck_name = 2;
  double latest_price= 3;
  string message = 4;
}

message StockReply {
  int32 code = 1;
  string body = 2;
}
EOF

# 生产grpc代码
python -m grpc_tools.protoc -I. --python_out=. --grpc_python_out=. StockNotify.proto

In [ ]:
import grpc
import StockNotify_pb2
import StockNotify_pb2_grpc

def stock_notify(stock_code, stocck_name, latest_price, message="没啥事"):
    channel = grpc.insecure_channel('172.31.128.1:50051')
    stub = StockNotify_pb2_grpc.GreeterStub(channel)
    
    request = StockNotify_pb2.StockRequest(stock_code=stock_code, 
                                           stocck_name=stocck_name,
                                           latest_price=latest_price,
                                           message=message)
    
    response = stub.StockNotify(request)
    print(f"gRPC 调用结果： {response.body}")

# stock_notify(stock_code="xxx", stocck_name="xxxx", latest_price=9.0)

In [2]:
# 下载日k数据
from functools import reduce
import requests
import json


def get_stock_kline_url(stock_code):
    base_url = 'https://push2his.eastmoney.com/api/qt/stock/kline/get'
    query_params = {
        "secid": f"{stock_code}",
        "ut": "fa5fd1943c7b386f172d6893dbfba10b",
        "fields1": "f1,f2,f3,f4,f5,f6",
        "fields2": "f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61",
        "klt": "101",
        "fqt": "1",
        "beg": "0",
        "end": "20500101",
        "lmt": "120",
        }
    query_string = reduce(lambda x,y:str(x)+'&'+str(y), map(lambda x:str(x)+'='+str(query_params.get(x, '')), query_params))
    return f"{base_url}?{query_string}"

def get_kline_data(stock_code):
    url = get_stock_kline_url(stock_code)
    response = requests.get(url)
    message = response.content
    if message:
        message = message.decode('utf-8').strip()
    else:
        raise Exception(f"{stock_code}获取k线失败")
    with open(f'{stock_code}_kline.json', 'w+') as f:
        f.write(message)
    return None

def get_latest_20_close_price(stock_code):
    with open(f'{stock_code}_kline.json', 'r') as f:
        raw_data = json.load(f)

    data = raw_data.get('data', {}) 
    if not data:
        raise Exception(f"文件{stock_code}_kline.json中data数据为空")
    
    kline_data = data.get('klines', [])
    if not kline_data:
        raise Exception(f"{stock_code}解析klines失败")

    latest_20_kline = kline_data[-20:]
    return [ float(line.split(",")[2])
        for line in latest_20_kline
    ]


In [ ]:
import asyncio
import aiohttp
from functools import reduce
import json
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import signal
import threading
import time
from IPython.display import display, clear_output


font_dirs = '/usr/share/fonts'
font_files = matplotlib.font_manager.findSystemFonts(fontpaths=font_dirs)
for font_file in font_files:
    matplotlib.font_manager.fontManager.addfont(font_file)
matplotlib.rcParams['font.family'] = 'WenQuanYi Zen Hei'
matplotlib.rcParams['axes.unicode_minus'] = False


def get_stock_url(stock_code):
    base_url = 'https://29.push2.eastmoney.com/api/qt/stock/trends2/sse'
    query_params = {
        "fields1": "f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f17",
        "fields2": "f51,f52,f53,f54,f55,f56,f57,f58",
        "mpi": "1000",
        "ut": "fa5fd1943c7b386f172d6893dbfba10b",
        "secid": f"{stock_code}",
        "ndays": 1,
        "iscr": 0,
        "iscca": 0,
        "wbp2u": "|0|0|0|web"
        }
    query_string = reduce(lambda x,y:str(x)+'&'+str(y), map(lambda x:str(x)+'='+str(query_params.get(x, '')), query_params))
    url = f"{base_url}?{query_string}"
   
    return url

def get_url_headers(stock_code):
    stock_exchange = {
        "0": "sz",
        "1": "sh"
        }
    ex_key, code = stock_code.split(".")
    headers = {
        'Accept': 'text/event-stream',
        'Accept-Language':  'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
        'Cache-Control':  'no-cache',
        'Connection':  'keep-alive',
        'Origin':  'https://quote.eastmoney.com',
        'Referer':  f'https://quote.eastmoney.com/{stock_exchange[ex_key]}{code}.html',
        'Sec-Fetch-Dest':  'empty',
        'Sec-Fetch-Mode':  'cors',
        'Sec-Fetch-Site':  'same-site',
        'User-Agent':  'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36 Edg/133.0.0.0',
        'sec-ch-ua':  '"Not(A:Brand";v="99", "Microsoft Edge";v="133", "Chromium";v="133"',
        'sec-ch-ua-mobile':  '?0',
        'sec-ch-ua-platform':  '"Windows"'
        }
    return headers

def init_stock_data(stock_code, **kwargs):
    global stock_datas
    name = kwargs.get("name")
    if not name:
        return
    stock_datas[stock_code] = {
        "name": name,
        "trad_time": [],
        "open_price":[],
        "close_price":[],
        "average_price":[]
    }

def update_stock_data(stock_code, **kwargs):
    global stock_datas
    for k in ['open_price', 'close_price', 'average_price']:
        stock_datas[stock_code][k].append(float(kwargs.get(k, 0)))
    stock_datas[stock_code]['trad_time'].append(kwargs.get('trad_time', ''))
    # print(f"stock_datas:{stock_datas}")
    

def parse_data(stock_code, data):
    if not data:
        return
    data = data[5:]
    data = json.loads(data)
    resp_data = data.get("data", {}) or {}
    if not resp_data:
        return 
    
    stock_name = resp_data.get("name")
    if stock_name:
        init_stock_data(stock_code=stock_code, name=stock_name)
    trends_datas = resp_data.get("trends")
    if not trends_datas:
        print("暂时没有trends数据")
        return
    
    lastest_trend_data = trends_datas[-1]
    print(f"lastest_trend_data:{lastest_trend_data}")
    trad_time, open_price, close_price, high_price, \
        low_price, trad_volume, trad_amount, \
            average_price = lastest_trend_data.split(',')
    # print(f"{trad_time} {close_price}")
    update_stock_data(
        stock_code=stock_code, trad_time=trad_time, open_price=open_price, 
        close_price=close_price, high_price=high_price, low_price=low_price, 
        trad_volume=trad_volume, trad_amount=trad_amount, average_price=average_price
        )
        
        
async def get_stock_data(stock_code):
    url = get_stock_url(stock_code)
    headers = get_url_headers(stock_code)
    try:
        timeout = aiohttp.ClientTimeout(total=None, connect=None, sock_connect=None, sock_read=None)
        async with aiohttp.ClientSession(timeout=timeout) as session:
            async with session.get(url, headers=headers) as response:
                if response.status == 200:
                    async for line in response.content:
                        if line:
                            message = line.decode('utf-8').strip()
                            print('fetch data...')
                            if message:
                                parse_data(stock_code, message)
                        if stop_event.is_set():
                            print(f"get_stock_data: {stock_code} is exiting...")
                            raise Exception(f"exit {stock_code} data")
                else:
                    print(f"Failed to connect to {url}, Status code: {response.status}")
                
    except Exception as e:
        print(f"Error while connecting to {stock_code}: {e}")
        # raise e


def get_stocks(stock_codes):
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    tasks = [get_stock_data(code) for code in stock_codes]
    loop.run_until_complete(asyncio.gather(*tasks))


def display_data(stock_codes):
    global stock_datas
    fig, axes = plt.subplots(len(stock_codes), 1)
    for i,code in enumerate(stock_codes):
        st_data = stock_datas.get(code)
        if not st_data:
            continue
        name, close_price, trad_time = st_data["name"], st_data["close_price"], st_data["trad_time"]
        if not close_price:
            continue
        notify_policy(stock_code=code,
                      stock_name=name,
                      trad_time=trad_time[-1],
                      latest_price=close_price[-1])
        axes[i].plot(range(len(close_price)), close_price, 'r-', label='close')
        axes[i].set_title(f"当前{trad_time[-1]}股票{name}价格{close_price[-1]}")
        axes[i].legend()

    plt.tight_layout()
    plt.show()

def stock_show(stock_codes):
    while not stop_event.is_set():
        clear_output(wait=True)
        print("display")
        display_data(stock_codes)
        time.sleep(2)
    else:
        print("exit show...")

def rule_1(stock_code, stock_name, trad_time, latest_price):
    """
    当股价超过up_limit时，股价偏高
    """
    global stock_policy
    if not stock_policy.get(stock_code):
        return
    up_limit = stock_policy.get(stock_code, {}).get("up")
    if latest_price >= up_limit:
        message = f"""
        当前{trad_time}股票{stock_name}价格超过{up_limit},
        可能过高，提醒卖出
        """
        stock_notify(stock_code=stock_code,
                     stocck_name=stock_name,
                     latest_price=latest_price,
                     message=message)
    
def rule_2(stock_code, stock_name, trad_time, latest_price):
    """
    当股价低于down_limit时，股价偏低
    """
    global stock_policy
    if not stock_policy.get(stock_code):
        return
    down_limit = stock_policy.get(stock_code, {}).get("down")
    if latest_price <= down_limit:
        message = f"""
        当前{trad_time}股票{stock_name}价格低于{down_limit},
        可能过低，提醒买入
        """
        stock_notify(stock_code=stock_code,
                     stocck_name=stock_name,
                     latest_price=latest_price,
                     message=message)

def rule_3(stock_code, stock_name, trad_time, latest_price):
    """
    当股价穿过MA20时，可能形成买卖点
    """
    global stock_policy
    if not stock_policy.get(stock_code):
        return
    
    ma20_rule = stock_policy.get(stock_code, {}).get("ma20")
    if not ma20_rule:
        return
    
    latest_20 = get_latest_20_close_price(stock_code)
    latest_20[-1] = latest_price
    ma20 = sum(latest_20)/len(latest_20)
    print(f"{stock_name} MA20:{ma20}")
    if ma20_rule == '>' and latest_price >= ma20:
        message = f"""
        当前{trad_time}股票{stock_name}价格上穿MA20值：{ma20},
        可能形成上升趋势，提醒买入
        """
        stock_notify(stock_code=stock_code, 
                 stocck_name=stock_name,
                 latest_price=latest_price,
                 message=message)

    if ma20_rule == '<' and latest_price <= ma20:
        message = f"""
        当前{trad_time}股票{stock_name}价格下穿MA20值：{ma20},
        可能形成下降趋势，提醒卖出
        """
        stock_notify(stock_code=stock_code, 
                 stocck_name=stock_name,
                 latest_price=latest_price,
                 message=message)

def rule_4(stock_code, stock_name, trad_time, latest_price):
    """
    计算20日、10日、5日、3日的k线斜率，计算峰点或者谷点的位置，预测见顶或者触底
    """
    global stock_policy
    if not stock_policy.get(stock_code):
        return
    
    ma20_rule = stock_policy.get(stock_code, {}).get("ma20")
    if not ma20_rule:
        return
    
    latest_20 = get_latest_20_close_price(stock_code)
    latest_20[-1] = latest_price
    
    slop_20, _ = np.polyfit(range(20), latest_20, 1)
    slop_10, _ = np.polyfit(range(10), latest_20[-10:], 1)
    slop_5, _ = np.polyfit(range(5), latest_20[-5:], 1)
    slop_3, _ = np.polyfit(range(3), latest_20[-3:], 1)
    if slop_3 > slop_5:
        # 短期上升趋势
        pass
    if slop_3 < slop_5:
        # 短期下降趋势
        pass
    

def notify_policy(stock_code, stock_name, trad_time, latest_price):
    rule_3(stock_code, stock_name, trad_time, latest_price)
    rule_2(stock_code, stock_name, trad_time, latest_price)
    rule_1(stock_code, stock_name, trad_time, latest_price)


def signal_handler(sig, frame):
    print("Signal received:", sig)
    print("Shutting down...")
    # 在接收到信号后，设置一个标志变量来停止所有线程
    stop_event.set()
    

"""
中公教育： 0.002607
洛阳钼业： 1.603993
"""
stock_codes = ["0.002607", "1.603993"]
stock_policy = {
    "0.002607": {"up": 3.9, "down": 3.3, "ma20": ">"},
    # "1.603993": {"up": 7.44, "down": 6.8, "ma20": ">"}
    "1.603993": {"up": 7.44, "down": 6.8, "ma20": "="}
}
# stock_data = {"code": {"name":"", "open_price":[], "close_price":[], "average_price":[]}}
stock_datas = {}
for code in stock_codes:
    print(f"获取{code}的k线值")
    get_kline_data(code)

signal.signal(signal.SIGINT, signal_handler)
signal.signal(signal.SIGTERM, signal_handler)

stop_event = threading.Event()

# 创建两个线程
thread1 = threading.Thread(target=get_stocks, args=(stock_codes,))
thread2 = threading.Thread(target=stock_show, args=(stock_codes,))

# 启动线程
thread1.start()
thread2.start()

# 等待线程执行完毕
thread1.join()
thread2.join()
